A linear model attempts to estimate sets of observables $y$ with linear combinations of predictors $x$.

---

If $x$ has a single dimension, i.e. there is only one coefficient of linearity, the obtained model is univariate.

If, on the other hand, $x$ has more than one dimension, i.e. each of its dimensions has a corresponding linear coefficient, the obtained model is multivariate.

---

Additionally, the $x$ and $y$ sets may be affected by uncertainties.

Any uncertainty affecting any quantity is here assumed to stem from a normal distribution.

---

This notebook attempts to summarise the best `python` tools with which each of these cases can be better tackled.



# Import statements

In [1]:
import numpy as np

from scipy import odr, stats
from sklearn import linear_model
from statsmodels.regression.linear_model import OLS, WLS
from statsmodels.tools import tools

# Univariate models

In [2]:
n = 100
p = 2   # number of parameters to fit, i.e. slope and intercept

real_slope = -5.3
real_intercept = 10.7

x_sigma = 0.5
y_sigma = 3.5

# create column vector of predictors
x = np.linspace(start=-30, stop=20, num=n) + np.random.normal(loc=0, scale=x_sigma, size=(n,))
x = x.reshape(-1, 1)

x_design = tools.add_constant(x)

# create column vector of observables
y = real_slope * np.linspace(start=-30, stop=20, num=n) + real_intercept + np.random.normal(loc=0, scale=y_sigma, size=(n,))
y = y.reshape(-1, 1)

x_to_predict = 6
x_to_predict_col_vec = np.array([[1],[x_to_predict]])

alpha = 1-.95

### Without uncertainties

##### Using `scipy.stats`

In [3]:
# vectors need to be converted to rows
x_row = x.flatten()
y_row = y.flatten()

sp_result = stats.linregress(x.flatten(), y_row)

print(f"Estimated slope {sp_result.slope:.4f} ± {sp_result.stderr:.4f}\nEstimated intercept {sp_result.intercept:.4f} ± {sp_result.intercept_stderr:.4f}")

Estimated slope -5.2978 ± 0.0320
Estimated intercept 10.5699 ± 0.4925


In [4]:
residuals = y_row - (x_row * sp_result.slope + sp_result.intercept)
s_value = np.linalg.norm(residuals, 2) / np.sqrt(n - p)

y_estimated = x_to_predict * sp_result.slope + sp_result.intercept

inner_sqrt_term = (x_to_predict_col_vec.T @ np.linalg.inv(x_design.T @ x_design ) @ x_to_predict_col_vec )[0,0]

one_sided_CL_magnitude = stats.t.ppf(q = 1-(alpha/2), df = n-p) * s_value * np.sqrt(inner_sqrt_term)

print(f"95% Confidence level at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_CL_magnitude:.4f}")


95% Confidence level at x = 6 is y_estimated = -21.2167 ± 1.1586


In [5]:
one_sided_PL_magnitude = stats.t.ppf(q = 1-alpha/2, df = n-p) * s_value * np.sqrt(1+inner_sqrt_term)

print(f"95% Prediction interval at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_PL_magnitude:.4f}")

95% Prediction interval at x = 6 is y_estimated = -21.2167 ± 9.3133


##### Using `sklearn`

In [6]:
sk_lm = linear_model.LinearRegression()
sk_result = sk_lm.fit(x, y)

print(f"Estimated slope {sk_result.coef_[0,0]:.4f} \nEstimated intercept {sk_result.intercept_[0]:.4f} ")

Estimated slope -5.2978 
Estimated intercept 10.5699 


In [7]:
residuals = y_row - (x_row * sk_result.coef_[0,0] + sk_result.intercept_[0])
s_value = np.linalg.norm(residuals, 2) / np.sqrt(n - p)

y_estimated = x_to_predict * sk_result.coef_[0,0] + sk_result.intercept_[0]

inner_sqrt_term = (x_to_predict_col_vec.T @ np.linalg.inv(x_design.T @ x_design ) @ x_to_predict_col_vec )[0,0]

one_sided_CL_magnitude = stats.t.ppf(q = 1-(alpha/2), df = n-p) * s_value * np.sqrt(inner_sqrt_term)

print(f"95% Confidence level at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_CL_magnitude:.4f}")

95% Confidence level at x = 6 is y_estimated = -21.2167 ± 1.1586


In [8]:
one_sided_PL_magnitude = stats.t.ppf(q = 1-alpha/2, df = n-p) * s_value * np.sqrt(1+inner_sqrt_term)

print(f"95% Prediction interval at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_PL_magnitude:.4f}")

95% Prediction interval at x = 6 is y_estimated = -21.2167 ± 9.3133


##### Using `statsmodels`

In [10]:
sm_lm = OLS(y, x_design)
sm_result = sm_lm.fit()
# print(sm_result.summary())

In [11]:
prediction = sm_result.get_prediction(x_to_predict_col_vec.T)

y_estimated = prediction.predicted_mean[0]

one_sided_CL_magnitude = prediction.summary_frame(alpha=alpha).mean_ci_upper.iloc[0] - prediction.predicted_mean[0]

print(f"95% Confidence level at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_CL_magnitude:.4f}")

95% Confidence level at x = 6 is y_estimated = -21.2167 ± 1.1586


In [12]:
one_sided_PL_magnitude = prediction.summary_frame(alpha=alpha).obs_ci_upper.iloc[0] - prediction.predicted_mean[0]

print(f"95% Prediction interval at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_PL_magnitude:.4f}")

95% Prediction interval at x = 6 is y_estimated = -21.2167 ± 9.3133


##### Conclusion

Use `statsmodels` since it is neater and there is less variable manipulation.

Do not use `scikit-learn` since it doesn't even provide standard errors for the fit coefficients.

### With uncertainties in $y$

Here, it is assumed that there is no dependence of $\Delta y$ on the value of $y$ itself and that each $y$ measurement has independent $\Delta y$

In [13]:
y_weights = 1/abs(np.random.normal(loc=y_sigma, scale=y_sigma/10, size=(n,)))**2

##### Using `sklearn`

In [14]:
sk_weighted_lm = linear_model.LinearRegression()
sk_weighted_result = sk_weighted_lm.fit(x, y, sample_weight=y_weights)

print(f"Estimated slope {sk_weighted_result.coef_[0,0]:.4f} \nEstimated intercept {sk_weighted_result.intercept_[0]:.4f} ")

Estimated slope -5.3048 
Estimated intercept 10.6341 


In [15]:
residuals = y_row - (x_row * sk_weighted_result.coef_[0,0] + sk_weighted_result.intercept_[0])
s_squared_value = sum(y_weights * residuals**2) / (n - p)

y_estimated = x_to_predict * sk_weighted_result.coef_[0,0] + sk_weighted_result.intercept_[0]

inner_sqrt_term = (x_to_predict_col_vec.T @ np.linalg.inv(x_design.T @ np.diag(y_weights) @ x_design ) @ x_to_predict_col_vec )[0,0]

one_sided_CL_magnitude = stats.t.ppf(q = 1-(alpha/2), df = n-p) * np.sqrt(s_squared_value * inner_sqrt_term)

print(f"95% Confidence level at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_CL_magnitude:.4f}")

95% Confidence level at x = 6 is y_estimated = -21.1944 ± 1.1554


In [16]:
one_sided_PL_magnitude = stats.t.ppf(q = 1-alpha/2, df = n-p) * np.sqrt(s_squared_value + s_squared_value * inner_sqrt_term)

print(f"95% Prediction interval at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_PL_magnitude:.4f}")

95% Prediction interval at x = 6 is y_estimated = -21.1944 ± 2.9009


##### Using `statsmodels`

In [17]:
sm_weighted_lm = WLS(y, x_design, y_weights )
sm_weighted_result = sm_weighted_lm.fit()
# print(sm_weighted_result.summary())

In [18]:
weighted_prediction = sm_weighted_result.get_prediction(x_to_predict_col_vec.T)

y_estimated = weighted_prediction.predicted_mean[0]

one_sided_CL_magnitude = weighted_prediction.summary_frame(alpha=alpha).mean_ci_upper.iloc[0] - weighted_prediction.predicted_mean[0]

print(f"95% Confidence level at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_CL_magnitude:.4f}")

95% Confidence level at x = 6 is y_estimated = -21.1944 ± 1.1554


In [19]:
one_sided_PL_magnitude = weighted_prediction.summary_frame(alpha=alpha).obs_ci_upper.iloc[0] - weighted_prediction.predicted_mean[0]

print(f"95% Prediction interval at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_PL_magnitude:.4f}")

95% Prediction interval at x = 6 is y_estimated = -21.1944 ± 2.9009


##### Conclusion

Again, the `statsmodels` approach is neater and preferred.

`sklearn` is also fine to be used, but some manipulation needs to be done with the variables.

Do not use `scipy.odr` since it does not perform an exact weighted least squares calculation to estimate the fit parameters and instead uses the Levenberg-Marquardt-type algorithm.

### With uncertainties in $x$ and $y$

In [ ]:
x_weights = 1/abs(np.random.normal(loc=x_sigma, scale=x_sigma/10, size=(n,)))**2

##### Using `scipy.odr`

In [ ]:
def univariate_linear_function(params, x):
    """
    Univariate linear calculation
    Input: params (list) contain the linear coefficients
    Input: x (row vector) contains the independent variables
    Output: the linear calculation
    """

    return params[0] + params[1] * x

In [ ]:
odr_model = odr.Model(univariate_linear_function)

odr_data = odr.RealData(x=x.flatten(), y=y.flatten(), sx=1/np.sqrt(x_weights), sy=1/np.sqrt(y_weights))

odr_instance = odr.ODR(odr_data, odr_model, beta0=[1,30])

odr_output = odr_instance.run()

odr_output.pprint()

##### Conclusion

# Multivariate models

In [20]:
n = 1000
p = 4   # number of parameters to fit, i.e. slopes and intercept

real_slope_a = -5.3
real_slope_b = .3
real_slope_c = 50

real_intercept = -5.7

x_sigma_a = 0.5
x_sigma_b = 0.5
x_sigma_c = 0.5
y_sigma = 3.5

# create design matrix with intercept column

x_design = np.ones(shape=(n,p))
x_design[:,1] = np.linspace(start=-30, stop=20, num=n) + np.random.normal(loc=0, scale=x_sigma_a, size=(n,))
x_design[:,2] = np.linspace(start=30, stop=120, num=n) + np.random.normal(loc=0, scale=x_sigma_b, size=(n,))
x_design[:,3] = np.linspace(start=0, stop=7, num=n) + np.random.normal(loc=0, scale=x_sigma_c, size=(n,))

# create column vector of observables
y =  x_design[:,1] * real_slope_a + x_design[:,2] * real_slope_b + x_design[:,3] * real_slope_c + real_intercept + np.random.normal(loc=0, scale=y_sigma, size=(n,))
y = y.reshape(-1, 1)

x_to_predict_col_vec = np.array([[1], [6], [100], [5]])

alpha = 1-.95

### Without uncertainties

##### Using `sklearn`

In [21]:
sk_lm = linear_model.LinearRegression(fit_intercept=False)
sk_result = sk_lm.fit(x_design, y)

print(f"Estimated slopes {sk_result.coef_[0,1:]} \nEstimated intercept {sk_result.coef_[0,0]:.4f} ")

Estimated slopes [-5.21803685  0.23886815 50.27246217] 
Estimated intercept -1.6310 


In [22]:
residuals = y - x_design @ sk_result.coef_.T
s_value = np.linalg.norm(residuals, 2) / np.sqrt(n - p)

y_estimated = (x_to_predict_col_vec.T @ sk_result.coef_.T).item()

inner_sqrt_term = (x_to_predict_col_vec.T @ np.linalg.inv(x_design.T @ x_design ) @ x_to_predict_col_vec )[0,0]

one_sided_CL_magnitude = stats.t.ppf(q = 1-(alpha/2), df = n-p) * s_value * np.sqrt(inner_sqrt_term)

print(f"95% Confidence level at x = {x_to_predict_col_vec.T[0, 1:]} is y_estimated = {y_estimated:.4f} ± {one_sided_CL_magnitude:.4f}")

95% Confidence level at x = [  6 100   5] is y_estimated = 242.3099 ± 1.1358


In [23]:
one_sided_PL_magnitude = stats.t.ppf(q = 1-alpha/2, df = n-p) * s_value * np.sqrt(1+inner_sqrt_term)

print(f"95% Prediction interval at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_PL_magnitude:.4f}")

95% Prediction interval at x = 6 is y_estimated = 242.3099 ± 6.8576


##### Using `statsmodels`

In [24]:
sm_lm = OLS(y, x_design)
sm_result = sm_lm.fit()
# print(sm_result.summary())

In [25]:
prediction = sm_result.get_prediction(x_to_predict_col_vec.T)

y_estimated = prediction.predicted_mean[0]

one_sided_CL_magnitude = prediction.summary_frame(alpha=alpha).mean_ci_upper.iloc[0] - prediction.predicted_mean[0]

print(f"95% Confidence level at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_CL_magnitude:.4f}")

95% Confidence level at x = 6 is y_estimated = 242.3099 ± 1.1358


In [26]:
one_sided_PL_magnitude = prediction.summary_frame(alpha=alpha).obs_ci_upper.iloc[0] - prediction.predicted_mean[0]

print(f"95% Prediction interval at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_PL_magnitude:.4f}")

95% Prediction interval at x = 6 is y_estimated = 242.3099 ± 6.8576


##### Conclusion

The `statsmodels` module seems to provide more robust machinery.

`scipy` does not provide simple multivariate linear regression. `scipy.odr` does, but its optimisation algorithms do not calculate exact solution for this least squares problem.

### With uncertainties in $y$

Here, it is assumed that there is no dependence of $\Delta y$ on the value of $y$ itself and that each $y$ measurement has independent $\Delta y$

In [27]:
y_weights = 1/abs(np.random.normal(loc=y_sigma, scale=y_sigma/10, size=(n,)))**2

y_weights = y_weights.reshape(-1, 1)

##### Using `sklearn`

In [28]:
sk_weighted_lm = linear_model.LinearRegression(fit_intercept=False)
sk_weighted_result = sk_weighted_lm.fit(x_design, y, sample_weight=(y_weights.T)[0])

print(f"Estimated slopes {sk_weighted_result.coef_[0,1:]} \nEstimated intercept {sk_weighted_result.coef_[0,0]:.4f} ")

Estimated slopes [-5.22345038  0.24319559 50.26651432] 
Estimated intercept -1.9860 


In [29]:
residuals = y - x_design @ sk_weighted_result.coef_.T
s_squared_value = (sum(y_weights * residuals**2) / (n - p)).item()

y_estimated = (x_to_predict_col_vec.T @ sk_weighted_result.coef_.T).item()

inner_sqrt_term = (x_to_predict_col_vec.T @ np.linalg.inv(x_design.T @ np.diag(y_weights.T[0]) @ x_design) @ x_to_predict_col_vec ).item()

one_sided_CL_magnitude = stats.t.ppf(q = 1-(alpha/2), df = n-p) * np.sqrt(s_squared_value * inner_sqrt_term)

print(f"95% Confidence level at x = {x_to_predict_col_vec.T[0, 1:]} is y_estimated = {y_estimated:.4f} ± {one_sided_CL_magnitude:.4f}")

95% Confidence level at x = [  6 100   5] is y_estimated = 242.3254 ± 1.1375


In [31]:
one_sided_PL_magnitude = stats.t.ppf(q = 1-alpha/2, df = n-p) * np.sqrt(s_squared_value + s_squared_value * inner_sqrt_term)

print(f"95% Prediction interval at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_PL_magnitude:.4f}")

95% Prediction interval at x = 6 is y_estimated = 242.3254 ± 2.2684


##### Using `statsmodels`

In [34]:
sm_weighted_lm = WLS(y, x_design, y_weights )
sm_weighted_result = sm_weighted_lm.fit()
# print(sm_weighted_result.summary())

In [38]:
prediction = sm_weighted_result.get_prediction(x_to_predict_col_vec.T)

y_estimated = prediction.predicted_mean[0]

one_sided_CL_magnitude = prediction.summary_frame(alpha=alpha).mean_ci_upper.iloc[0] - prediction.predicted_mean[0]

print(f"95% Confidence level at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_CL_magnitude:.4f}")

95% Confidence level at x = 6 is y_estimated = 242.3254 ± 1.1375


In [39]:
one_sided_PL_magnitude = prediction.summary_frame(alpha=alpha).obs_ci_upper.iloc[0] - prediction.predicted_mean[0]

print(f"95% Prediction interval at x = {x_to_predict} is y_estimated = {y_estimated:.4f} ± {one_sided_PL_magnitude:.4f}")

95% Prediction interval at x = 6 is y_estimated = 242.3254 ± 2.2684


##### Conclusion